In [ ]:
! pip install datasets transformers rouge-score nltk py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 10.7 MB/s 
     |████████████████████████████████| 5.8 MB 23.5 MB/s 
     |████████████████████████████████| 65 kB 2.0 MB/s 
     |████████████████████████████████| 182 kB 66.6 MB/s 
     |████████████████████████████████| 132 kB 71.9 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
     |████████████████████████████████| 127 kB 78.1 MB/s 
     |████████████████████████████████| 7.6 MB 55.4 MB/s 
     |████████████████████████████████| 2.3 MB 30.4 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 378 kB 68.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 357 kB 70.1 MB/s 
     |████████████████████████████████| 139 kB 76.4 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NLP Project

/content/drive/MyDrive/NLP Project


# Fine-tuning a model on a summarization task

## Loading the dataset

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,AutoTokenizer


In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-5-a2ef1525156b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


#**Compute Metices**

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds] #my predictions from model
    
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]#original prediction

    
    for i in range(0,5):
      print(tokenized_datasets_test["dialogue"][i])
      print("-------------------------")
      print("------>Predictions by Model")
      print(decoded_preds[i])
      print("----->Predictions Original")
      print(decoded_labels[i])
      print("**************************")
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# **CUSTOM TRAINER**

In [ ]:
import torch.nn as nn
class CustomTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # How the loss is computed by Trainer. By default, all models return the loss in the first element.
        # Subclass and override for custom behavior.
      
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # print("outputs**********************: \n", outputs["last_hidden_state"].size())
        # print("logits*****************:", logits.view(-1, self.model.config.num_labels))
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
            loss_fct = nn.CrossEntropyLoss()
            cross_loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            loss += cross_loss
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        # Adding CrossEntropyLoss()
        # loss_fct = nn.CrossEntropyLoss()
        # cross_loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        # loss += cross_loss
        return (loss, outputs) if return_outputs else loss

## BART

### Preprocessing the data

In [ ]:
# model_checkpoint_500 = "/content/drive/MyDrive/NLP Project/test-dialogue-summarization/checkpoint-500"
# model_checkpoint_1000 = "/content/drive/MyDrive/NLP Project/test-dialogue-summarization/checkpoint-1000"
model_checkpoint = "facebook/bart-base"


In [ ]:
from transformers import AutoTokenizer
    
# tokenizer_500 = AutoTokenizer.from_pretrained(model_checkpoint_500)
# tokenizer_1000 = AutoTokenizer.from_pretrained(model_checkpoint_1000)
# tokenizer_1500 = AutoTokenizer.from_pretrained(model_checkpoint_1500)
# tokenizer_2000 = AutoTokenizer.from_pretrained(model_checkpoint_2000)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# tokenized_datasets_train = tokenized_datasets['train'].select(range(100))
# tokenized_datasets_val = tokenized_datasets['validation'].select(range(70))

tokenized_datasets_train = tokenized_datasets['train']
tokenized_datasets_val = tokenized_datasets['validation']
tokenized_datasets_test = tokenized_datasets['test']

### Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
batch_size = 32
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    # save_total_limit=10,
    num_train_epochs=5,
    logging_steps = 10, ## added
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
trainer = CustomTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,


In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, dialogue, id. If summary, dialogue, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 32


{'eval_loss': 1.5303208827972412,
 'eval_rouge1': 47.5105,
 'eval_rouge2': 24.1888,
 'eval_rougeL': 40.0868,
 'eval_rougeLsum': 43.8536,
 'eval_gen_len': 18.1125,
 'eval_runtime': 44.2729,
 'eval_samples_per_second': 18.476,
 'eval_steps_per_second': 0.587,
 'epoch': 5.0}

## T5

### Preprocessing the data

In [ ]:
model_checkpoint = "t5-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    task_prefix = "summarize: "
    inputs = examples["dialogue"]
    model_inputs = tokenizer([task_prefix + dialogue for dialogue in inputs], 
                             padding="max_length",
                             max_length=max_input_length, 
                             truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(examples["summary"], 
                        padding="max_length",
                        max_length=max_target_length, 
                        truncation=True)

    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in labels["input_ids"]]
        
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# sample a small set for development
# tokenized_datasets_train = tokenized_datasets['train'].select(range(100))
# tokenized_datasets_val = tokenized_datasets['validation'].select(range(70))


tokenized_datasets_train = tokenized_datasets['train']
tokenized_datasets_val = tokenized_datasets['validation']

### Fine-tuning the model

In [ ]:
# Parameters\
batch_size=8
training_args = Seq2SeqTrainingArguments(
    output_dir="t5_results",
    num_train_epochs=5,
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-4,
    warmup_steps=500,
    weight_decay=0.1,
    # label_smoothing_factor=0.1, ## causes to throw an error
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=10,
    # save_total_limit=3,
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# evaluate before training for comparison
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, dialogue, id. If summary, dialogue, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.411588191986084,
 'eval_rouge1': 24.5814,
 'eval_rouge2': 7.6806,
 'eval_rougeL': 20.811,
 'eval_rougeLsum': 22.5157,
 'eval_gen_len': 18.3191,
 'eval_runtime': 106.4884,
 'eval_samples_per_second': 7.682,
 'eval_steps_per_second': 0.967}

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, summary, dialogue. If id, summary, dialogue are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14732
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9210
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is 

Epoch,Training Loss,Validation Loss


Saving model checkpoint to t5_results/checkpoint-500
Configuration saved in t5_results/checkpoint-500/config.json
Model weights saved in t5_results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to t5_results/checkpoint-1000
Configuration saved in t5_results/checkpoint-1000/config.json
Model weights saved in t5_results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5_results/checkpoint-1500
Configuration saved in t5_results/checkpoint-1500/config.json
Model weights saved in t5_results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-15

NameError: ignored

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, dialogue, id. If summary, dialogue, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


{'eval_loss': 1.4014945030212402,
 'eval_rouge1': 48.1292,
 'eval_rouge2': 24.7742,
 'eval_rougeL': 40.5648,
 'eval_rougeLsum': 44.3401,
 'eval_gen_len': 17.0868,
 'eval_runtime': 108.3146,
 'eval_samples_per_second': 7.552,
 'eval_steps_per_second': 0.951,
 'epoch': 5.0}